In [ ]:
import numpy as np
import pandas as pd
import urllib.request
import json
from pandas import json_normalize
from urllib.parse import urlencode, quote_plus, unquote

In [ ]:
# API 호출을 위한 설정 준비 작업
api_link = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/'
api_key = '8F62E987CCE7CBBBB9E8DD159D3A37EE'
steamid = '76561198031695144' # 스트리머 멀럭킹
queryParams = '?' + urlencode({quote_plus('key') : api_key,
                               quote_plus('steamid') : steamid,
                               quote_plus('relationship') : 'friend'
                               })
url = api_link + queryParams

In [ ]:
#데이터를 요청하고 변환합니다.
text = urllib.request.urlopen(url).read().decode('utf-8')
json_return = json.loads(text)

In [ ]:
# JSON Response 확인
print(json_return)

{'friendslist': {'friends': [{'steamid': '76561197960421647', 'relationship': 'friend', 'friend_since': 1490008534}, {'steamid': '76561197966062591', 'relationship': 'friend', 'friend_since': 1595669725}, {'steamid': '76561197967479980', 'relationship': 'friend', 'friend_since': 1502453076}, {'steamid': '76561197971565071', 'relationship': 'friend', 'friend_since': 1423219650}, {'steamid': '76561197972650406', 'relationship': 'friend', 'friend_since': 1431962639}, {'steamid': '76561197973315893', 'relationship': 'friend', 'friend_since': 1461849261}, {'steamid': '76561197978049807', 'relationship': 'friend', 'friend_since': 1459594769}, {'steamid': '76561197986317848', 'relationship': 'friend', 'friend_since': 1502453066}, {'steamid': '76561197987498968', 'relationship': 'friend', 'friend_since': 1434712783}, {'steamid': '76561197987573026', 'relationship': 'friend', 'friend_since': 1635075281}, {'steamid': '76561197988813283', 'relationship': 'friend', 'friend_since': 1451056439}, {'s

In [ ]:
friends_list = {}
queue_user_to_scan = []

# 반환된 JSON에서 친구 목록만 가져오기
friends_data = json_return.get('friendslist').get('friends')

# 새 User의 친구 목록으로 작업
for friend in friends_data:
    if friend.get('steamid') not in friends_list:         # 해당 친구가 이미 스캔된 유저가 아닌 경우
        queue_user_to_scan.append(friend.get('steamid'))    # 해당 친구를 스캔 대상 큐에 추가
        friends_list[friend.get('steamid')] = 1             # 해당 친구를 친구 수집 목록에 추가
    else:                                                 # 해당 친구가 이미 스캔된 유저인 경우
        continue                                            # 스킵 후 계속 진행

In [ ]:
# Too many request를 피하기 위한 수동 Request
for i in range(20):
    user_scan()

In [ ]:
# 자료구조 상태 확인
print(friends_list)
print(queue_user_to_scan)
print(len(friends_list), len(queue_user_to_scan))

{'76561197960305875': '1', '76561197961691515': '1', '76561197962914411': '1', '76561197962961074': '1', '76561197963835346': '1', '76561197964136230': '1', '76561197964693285': '1', '76561197964962765': '1', '76561197965224637': '1', '76561197966246843': '1', '76561197967218127': '1', '76561197967558800': '1', '76561197967622273': '1', '76561197967913550': '1', '76561197968941709': '1', '76561197968950768': '1', '76561197969193820': '1', '76561197969315197': '1', '76561197969656712': '1', '76561197971891127': '1', '76561197971891881': '1', '76561197972182076': '1', '76561197973533254': '1', '76561197974326261': '1', '76561197974670103': '1', '76561197975320032': '1', '76561197979754835': '1', '76561197983325139': '1', '76561197987746602': '1', '76561197988022467': '1', '76561197988678465': '1', '76561197989453195': '1', '76561197989470511': '1', '76561197990251327': '1', '76561197990905503': '1', '76561197990974810': '1', '76561197990996428': '1', '76561197991575366': '1', '7656119799

In [ ]:
# Request 후 후처리 작업
def user_scan():
    # 큐에서 스캔 대상자 추출 후 쿼리 작성
    user_to_scan = queue_user_to_scan.pop(0)
    queryParams = '?' + urlencode({quote_plus('key') : api_key,
                                  quote_plus('steamid') : user_to_scan,
                                  quote_plus('relationship') : 'friend'
                                  })
    url = api_link + queryParams

    # 데이터를 요청하고 변환 : 스캔한 사용자 프로필이 Public이 아닌 경우 401 에러에 대한 예외 처리
    try:
        text = urllib.request.urlopen(url).read().decode('utf-8')
    except:
        text = '{}'
    json_return = json.loads(text)

    # 데이터 해석 작업
    if json_return != {}:
        # 반환된 JSON에서 친구 목록만 가져오기
        friends_data = json_return.get('friendslist').get('friends')

        # 새 User의 친구 목록으로 작업
        for friend in friends_data:
            if friend.get('steamid') not in friends_list:         # 해당 친구가 이미 스캔된 유저가 아닌 경우
                queue_user_to_scan.append(friend.get('steamid'))    # 해당 친구를 스캔 대상 큐에 추가
                friends_list[friend.get('steamid')] = 1             # 해당 친구를 친구 수집 목록에 추가
            else:                                                 # 해당 친구가 이미 스캔된 유저인 경우
                continue                                            # 스킵 후 계속 진행

In [ ]:
# 추출한 계정 값들이 유니크한지 검증
my_friends = friends_list.keys()
list_my_friends = list(set(my_friends))
print(list_my_friends)
print(len(list_my_friends))

['76561198114996151', '76561198084090170', '76561198124275837', '76561198123336224', '76561197980947088', '76561197993375934', '76561198364675974', '76561198327972815', '76561198091908131', '76561198045778710', '76561198122848635', '76561198039277946', '76561198834587260', '76561198118899137', '76561198004451860', '76561198012942388', '76561199073478393', '76561198016159785', '76561198338265116', '76561198021742316', '76561198110645006', '76561198009891312', '76561197990905503', '76561198138364501', '76561198077045174', '76561198848618122', '76561198053475616', '76561198084155996', '76561198352412418', '76561198823440764', '76561198004631882', '76561198093634040', '76561198140663827', '76561198051658334', '76561198008272345', '76561198047480434', '76561198075434760', '76561198068480070', '76561199011748832', '76561198086614413', '76561197979754835', '76561198049059254', '76561198076559468', '76561198121707649', '76561198156897580', '76561198016940778', '76561198371551559', '76561198045

In [ ]:
# 지속 추출을 위한 리스트&딕셔너리 임시 저장
import csv

with open("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Friends.csv", 'w') as file:
    writer = csv.writer(file)
    for k, v in friends_list.items():
        writer.writerow([k, v])
with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Queue.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(queue_user_to_scan)

In [ ]:
# 지속 추출 테스트용 리스트&딕셔너리 초기화

friends_list = {}
queue_user_to_scan = []

In [ ]:
# 지속 추출을 위한 리스트&딕셔너리 불러오기
import csv

friends_list = {}
queue_user_to_scan = []

with open("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Friends.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        friends_list[row[0]] = row[1]
with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Queue.csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        queue_user_to_scan = row

In [ ]:
# 데이터 병합 테스트
import csv

friends_list_A = {}
friends_list_B = []

with open("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Friends.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        friends_list[row[0]] = row[1]
with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/users2.csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        friends_list_B = row

In [ ]:
# 데이터 병합 전 로드 정상 여부 확인
friends_list_A_tolist = list(friends_list.keys())
print(len(friends_list_A_tolist))
print(len(friends_list_B))

10092
10627


In [ ]:
# 데이터 병합
friends_merged = friends_list_A_tolist + friends_list_B
friends_merged_unique = list(set(friends_merged))

In [ ]:
print(len(friends_merged))
print(len(friends_merged_unique))

20719
20711


In [ ]:
# 2번째 병합
friends_list_C = []

with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/users2-2.csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        friends_list_C = row

In [ ]:
# 데이터 병합 전 로드 정상 여부 확인
print(len(friends_list_C))

13758


In [ ]:
# 데이터 병합
friends_merged = friends_merged_unique + friends_list_C
friends_merged_unique = list(set(friends_merged))

In [ ]:
print(len(friends_merged))
print(len(friends_merged_unique))

34469
34325


In [ ]:
# 병합 완료된 데이터셋 저장
with open('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/UniqueFriends_3.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(list_chunked[2])

In [ ]:
# 3명 분으로 데이터 분할
def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

print("분할 전 : ", len(friends_merged_unique))

list_chunked = list_chunk(friends_merged_unique, 11500)
print("분할 후 : ", len(list_chunked))

분할 전 :  34325
분할 후 :  3


In [ ]:
# 분할 결과가 정상인지 내부 데이터 확인
print(len(list_chunked[2]))

11325
